# Preprocessing .mat for NWB

In [1]:
import numpy as np
import h5py
import os
import yaml
import pandas as pd
from datetime import datetime
import shutil
import importlib
import numpy as np
import pandas as pd
import LO_NWB_conv
import scipy.io


# Load data 

## .mat files

In [4]:
WRAO039 = "data/mouse_anastasia/WR/AO039_20190626.mat"  
WRAO027 = "data/mouse_anastasia/WR/AO027_20181101.mat"
WnonRAO084 = "data/mouse_anastasia/WnonR/AO084_20210507.mat"
WnonRAO086 = "data/mouse_anastasia/WnonR/AO086_20210617.mat"

### Rewarded

In [15]:

with h5py.File(WRAO039, 'r') as f:
    data_group = f['Data']
    data_ref = f["#refs#"]
    #print("Références trouvées :")
    #for key in data_ref.keys():
    #    print(f" - {key} → shape: {data_ref[key].shape}, dtype: {data_ref[key].dtype}")
    print("Contenu de 'Data' :")
    for key in data_group.keys():
        print(f" - {key} → shape: {data_group[key].shape}, dtype: {data_group[key].dtype}")


Contenu de 'Data' :
 - ARAindex → shape: (75, 1), dtype: float64
 - Area → shape: (75, 1), dtype: object
 - BaselineFR_Mean → shape: (75, 1), dtype: float64
 - BaselineFR_Session → shape: (75, 1), dtype: object
 - CRIndices → shape: (1, 583), dtype: uint8
 - ClusterCounter → shape: (75, 1), dtype: float64
 - EngagedTrials → shape: (583, 1), dtype: uint8
 - FAIndices → shape: (1, 583), dtype: uint8
 - HitIndices → shape: (1, 583), dtype: uint8
 - ISI_Violation → shape: (75, 1), dtype: float64
 - ISO_Distance → shape: (75, 1), dtype: float64
 - JawOnsetsTms → shape: (583, 1), dtype: float64
 - JawTrace → shape: (583, 1000), dtype: float64
 - LFPs → shape: (3, 1), dtype: object
 - LickData → shape: (482142, 1), dtype: float64
 - LickTime → shape: (482142, 1), dtype: float64
 - LightIndices → shape: (1, 583), dtype: float64
 - MDS → shape: (1, 1), dtype: float64
 - ML_DV_AP → shape: (75, 1), dtype: object
 - ML_DV_AP_32 → shape: (75, 1), dtype: object
 - MissIndices → shape: (1, 583), dtyp

### Non-rewarded

In [16]:

with h5py.File(WnonRAO086, 'r') as f:
    data_group = f['Data']
    data_ref = f["#refs#"]
    #print("Références trouvées :")
    #for key in data_ref.keys():
    #    print(f" - {key} → shape: {data_ref[key].shape}, dtype: {data_ref[key].dtype}")
    print("Contenu de 'Data' :")
    for key in data_group.keys():
        print(f" - {key} → shape: {data_group[key].shape}, dtype: {data_group[key].dtype}")

Contenu de 'Data' :
 - ARAindex → shape: (135, 1), dtype: float64
 - Area → shape: (135, 1), dtype: object
 - BaselineFR_Mean → shape: (135, 1), dtype: float64
 - CRIndices → shape: (424, 1), dtype: float64
 - ClusterCounter → shape: (135, 1), dtype: float64
 - CoilAmps → shape: (367, 1), dtype: float64
 - CoilOnsets → shape: (367, 1), dtype: float64
 - ISI_Violation → shape: (135, 1), dtype: float64
 - ISO_Distance → shape: (135, 1), dtype: float64
 - JawOnsets_Tms → shape: (317, 1), dtype: float64
 - JawOnsets_Tms_All → shape: (434, 1), dtype: float64
 - JawOnsets_Tms_CompleteLicks → shape: (372, 1), dtype: float64
 - JawTrace → shape: (1, 902787), dtype: float64
 - LFPs → shape: (3, 1), dtype: object
 - MDS → shape: (1, 1), dtype: float64
 - ML_DV_AP → shape: (135, 1), dtype: object
 - ML_DV_AP_32 → shape: (135, 1), dtype: object
 - MissIndices → shape: (367, 1), dtype: uint8
 - NoseSideTrace → shape: (1, 902787), dtype: float64
 - NoseTopTrace → shape: (1, 902787), dtype: float64
 

### Comparison


In [7]:
with h5py.File(WRAO039, 'r') as f:
	data_group = f['Data'] if 'Data' in f else f
	WRAO039 = {key: data_group[key][()] for key in data_group.keys()}

with h5py.File(WRAO027, 'r') as f:
	data_group = f['Data'] if 'Data' in f else f
	WRAO027 = {key: data_group[key][()] for key in data_group.keys()}

with h5py.File(WnonRAO084, 'r') as f:
	data_group = f['Data'] if 'Data' in f else f
	WnonRAO084 = {key: data_group[key][()] for key in data_group.keys()}

with h5py.File(WnonRAO086, 'r') as f:
	data_group = f['Data'] if 'Data' in f else f
	WnonRAO086 = {key: data_group[key][()] for key in data_group.keys()}

In [8]:
# Comparaison WRAO039 et WRAO027
keys_WRAO039 = set(WRAO039.keys())
keys_WRAO027 = set(WRAO027.keys())

same_keys = keys_WRAO039 == keys_WRAO027
print("Les deux dictionnaires ont les mêmes clés :", same_keys)

if not same_keys:
    print("Clés uniquement dans WRAO039 :", keys_WRAO039 - keys_WRAO027)
    print("Clés uniquement dans WRAO027 :", keys_WRAO027 - keys_WRAO039)

Les deux dictionnaires ont les mêmes clés : True


In [9]:
# Comparaison WnonRAO084 et WnonRAO086
keys_WnonRAO084 = set(WnonRAO084.keys())
keys_WnonRAO086 = set(WnonRAO086.keys())

same_keys = keys_WnonRAO084 == keys_WnonRAO086
print("Les deux dictionnaires ont les mêmes clés :", same_keys)

if not same_keys:
    print("Clés uniquement dans WnonRAO084 :", keys_WnonRAO084 - keys_WnonRAO086)
    print("Clés uniquement dans WnonRAO086 :", keys_WnonRAO086 - keys_WnonRAO084)

Les deux dictionnaires ont les mêmes clés : True


In [11]:
# Comaraison WR et WnonR
keys_WR = set(WRAO039.keys())
keys_WnonR = set(WnonRAO084.keys())
same_keys = keys_WR == keys_WnonR
print("Les deux dictionnaires ont les mêmes clés :", same_keys)

if not same_keys:
    print("Clés uniquement dans WR :", keys_WR - keys_WnonR)
    print("Clés uniquement dans WnonR :", keys_WnonR - keys_WR)

    identical_keys = keys_WR & keys_WnonR
    print("Clés présentes dans les deux dictionnaires :", identical_keys)


Les deux dictionnaires ont les mêmes clés : False
Clés uniquement dans WR : {'SpontLicksVideo_TrialOn', 'QuietW_Times', 'Whisking_Indices', 'SpontLicksPiezo_FakeTrialOn', 'StimIndices', 'LickData', 'SpontLicksVideo_FakeTrialOn', 'BaselineFR_Session', 'ReactionTimes', 'LickTime', 'HitIndices', 'Whisking_Times', 'SpontLicksPiezo_LickOn', 'Whisking_Indices_NoLick', 'Whisking_Times_NoLick', 'FAIndices', 'VideoOnsets', 'SpontLicksVideo_JawOn', 'EngagedTrials', 'LightIndices', 'threshold_amp', 'JawOnsetsTms'}
Clés uniquement dans WnonR : {'JawOnsets_Tms', 'JawOnsets_Tms_All', 'Whisking_Tms', 'ValveOnsets_Tms', 'RewardedLicks_Ind_MouseTriggered', 'JawOnsets_Tms_CompleteLicks', 'PiezoLickOnset_Tms', 'depth', 'RewardedLicks_Ind_Assosiation', 'Quiet_Tms', 'CoilAmps', 'Valve_Ind_Assosiation', 'VideoFrames_Tms', 'PiezoLickOnset_Tms_CompleteLicks', 'Whisking_Tms_NoLick', 'CoilOnsets', 'RewardedLicks_Ind_All', 'Quiet_Tms_NoLick', 'Valve_Ind_MouseTriggered'}
Clés présentes dans les deux dictionnaires

## Try anything .mat

In [6]:
WRAO039 = "data/mouse_anastasia/WR/AO039_20190626.mat"  
WRAO027 = "data/mouse_anastasia/WR/AO027_20181101.mat"
WnonRAO084 = "data/mouse_anastasia/WnonR/AO084_20210507.mat"
WnonRAO086 = "data/mouse_anastasia/WnonR/AO086_20210617.mat"

with h5py.File(WRAO027, 'r') as f:
	data_group = f['Data'] if 'Data' in f else f
	data = {key: data_group[key][()] for key in data_group.keys()}
	

In [11]:
print(data["TrialOnsets_All"].shape)
print(data["TrialOnsets_All"])
data["JawOnsetsTms"]

(402, 1)
[[  14.55696667]
 [  22.94666667]
 [  46.8768    ]
 [  54.9067    ]
 [  75.5665    ]
 [  84.63656667]
 [  90.92666667]
 [ 106.89666667]
 [ 118.87656667]
 [ 127.80636667]
 [ 151.5463    ]
 [ 158.91636667]
 [ 165.90663333]
 [ 176.35613333]
 [ 192.426     ]
 [ 203.28613333]
 [ 210.5563    ]
 [ 220.62636667]
 [ 230.2659    ]
 [ 244.58623333]
 [ 258.09626667]
 [ 273.696     ]
 [ 320.34336667]
 [ 330.01643333]
 [ 353.14613333]
 [ 360.08613333]
 [ 383.09606667]
 [ 390.64586667]
 [ 410.93596667]
 [ 425.23623333]
 [ 441.15616667]
 [ 451.51623333]
 [ 462.97603333]
 [ 481.17593333]
 [ 492.9858    ]
 [ 502.7462    ]
 [ 509.44186667]
 [ 517.30586667]
 [ 532.39606667]
 [ 543.54583333]
 [ 555.43033333]
 [ 576.376     ]
 [ 594.41626667]
 [ 612.17606667]
 [ 632.6056    ]
 [ 649.68546667]
 [ 657.47566667]
 [ 670.47543333]
 [ 677.4397    ]
 [ 702.75553333]
 [ 712.92993333]
 [ 732.0852    ]
 [ 740.6853    ]
 [ 750.69573333]
 [ 757.07536667]
 [ 764.19526667]
 [ 773.10533333]
 [ 781.67543333]
 [ 78

array([[          nan],
       [          nan],
       [          nan],
       [          nan],
       [          nan],
       [  84.76056667],
       [          nan],
       [          nan],
       [          nan],
       [          nan],
       [          nan],
       [ 159.04636667],
       [          nan],
       [ 176.47813333],
       [ 193.962     ],
       [ 203.41813333],
       [ 210.6783    ],
       [          nan],
       [          nan],
       [          nan],
       [          nan],
       [          nan],
       [          nan],
       [          nan],
       [          nan],
       [          nan],
       [          nan],
       [          nan],
       [          nan],
       [          nan],
       [          nan],
       [          nan],
       [          nan],
       [          nan],
       [          nan],
       [ 502.8802    ],
       [          nan],
       [          nan],
       [ 532.53006667],
       [          nan],
       [          nan],
       [ 577.878

# CSV file


In [ ]:

csv_data = pd.read_csv("data/Subject_Session_Selection.csv", sep=";")
csv_data.columns = csv_data.columns.str.strip() 
csv_data.columns

In [ ]:

mouse = ''.join(chr(c) for c in WnonRAO084['mouse'].flatten())
date = ''.join(chr(c) for c in WnonRAO084['date'].flatten())
session_name = f"{mouse}_{date}"  

try:
    subject_info = csv_data[csv_data['Session'].astype(str).str.strip() == session_name].iloc[0]
except IndexError:
    raise ValueError(f"Session {session_name} not found in the CSV file.")


 # TOTAL _-_-_-_-_-_-_-_
 

In [18]:
# Supprimer tous les fichiers dans le dossier NWB output
for file in os.listdir("data/nwb_output"):
    file_path = os.path.join("data/nwb_output", file)
    if os.path.isfile(file_path):
        os.remove(file_path)
print("Tous les fichiers ont été supprimés du dossier data/nwb_output.")
#data/mouse_anastasia/WnonR/AO084_20210507.mat
#data/mouse_anastasia/WR/AO027_20181101.mat
filename = "data/mouse_anastasia/WnonR/AO084_20210507.mat" # Chemin vers le fichier .mat
importlib.reload(LO_NWB_conv)
nwb_path = LO_NWB_conv.convert_data_to_nwb_an(mat_file=filename, output_folder="data/nwb_output", output_folder_config="data")

Tous les fichiers ont été supprimés du dossier data/nwb_output.
**************************************************************************
-_-_-_-_-_-_-_-_-_-_-_-_-_-_- NWB conversion _-_-_-_-_-_-_-_-_-_-_-_-_-_-_
 
📃 Creating config file for NWB conversion :
   - data/AO084_20210507_config.yaml
📑 Created NWB file :
ephys Whisker Non Rewarded:
     o 📌 Add general metadata
         - Subject metadata
         - Session metadata
         - Device metadata
         - Extracellular electrophysiology metadata
     o ⏸️ Add intervall container
     o 🧠 Add units container
     o ⚙️ Add processing container
         - Behavior data
         - No ephys data for AN sessions
         - Analysis complementary information
 
🔎 Validating NWB file before saving...
     o ✅ File is valid, no errors detected.
 
💾 Saving NWB file
     o 📂 NWB file saved at:
         - data/nwb_output/AO084_20210507_174652_2025_07_18.16-53-50.nwb
*************************************************************************

# Accessibilité 

In [ ]:
import h5py
from pynwb import NWBHDF5IO
import matplotlib.pyplot as plt
import numpy as np

# Ouvre le fichier NWB
nwb_file = "data/nwb_output/AO027_20181101_134512_2025_07_15.14-24-54.nwb"
io = NWBHDF5IO(nwb_file, mode='r')
nwb = io.read()

spike_times = nwb.units['spike_times'].data[:]  # spike times de tous les neurones
len(spike_times)





    

In [ ]:
behavior_events = nwb.processing['behavior'].data_interfaces['BehavioralEvents']
print("Contenu de BehavioralEvents :", list(behavior_events.fields.keys()))

In [ ]:
# Accède au conteneur interne
event_series = nwb.processing['behavior'].data_interfaces['BehavioralEvents'].time_series

# Affiche les noms des TimeSeries comportementales
print("Séries dans BehavioralEvents :", list(event_series.keys()))

In [ ]:
nwb.processing['behavior'].data_interfaces['BehavioralEvents'].time_series["TrialOnsets"].timestamps[:]  # Accède aux timestamps des événements "TrialOnsets"
print(nwb.processing['behavior'].data_interfaces['BehavioralEvents'].time_series["StimFlags"].data[:])